In [1]:
import csv
from selenium import webdriver
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge
from msedge.selenium_tools import EdgeOptions


# collecting tweets
def get_tweet(card):
    username = card.find_element("xpath", './div[1]//span').text
    handle = card.find_element("xpath", './div[1]//span[contains(text(),"@")]').text
    try:
        postdate = card.find_element("xpath", './div[1]//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    text = card.find_element("xpath", './div[2]/div[1]/div[1]').text
    reply_cnt = card.find_element("xpath", './/div[@data-testid="reply"]').text
    retweet_cnt = card.find_element("xpath", './/div[@data-testid="retweet"]').text
    like_cnt = card.find_element("xpath", './/div[@data-testid="like"]').text

    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet


# opening browser
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)

# login into tweeter
driver.get("https://www.twitter.com/login")
driver.maximize_window()
sleep(10)
username = driver.find_element("xpath", '//input[@name="text"]')
username.send_keys('Lalit14561844')
username.send_keys(Keys.RETURN)
sleep(5)
password = driver.find_element("xpath", '//input[@name="password"]')
password.send_keys('twitter@2002')
password.send_keys(Keys.RETURN)
sleep(3)




C:\Users\suraj\AppData\Local\Temp\ipykernel_1508\981734905.py:31: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(options=options)


In [16]:
# navigating to the trending page
search_input = driver.find_element("xpath", '//input[@aria-label="Search query"]')
search_input.send_keys('Queen Elizabeth')
search_input.send_keys(Keys.RETURN)
sleep(3)
driver.find_element("link text", "Latest").click()
sleep(3)

# all tweets are in here
data = []
tweet_ids = set()
last_pos = driver.execute_script("return window.pageYOffset;")
scrolling = True



In [17]:
while scrolling:
    page_cards = driver.find_elements("xpath", '//div[@class="css-1dbjc4n r-1iusvr4 r-16y2uox r-1777fci r-kzbkwu"]')
    for card in page_cards[-15:]:
        tweet = get_tweet(card)
        if tweet:
            id = ''.join(tweet)
            if id not in tweet_ids:
                tweet_ids.add(id)
                data.append(tweet)
    scroll_atmpt = 0
    while True:
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        sleep(3)
        curr_pos = driver.execute_script("return window.pageYOffset;")
        if last_pos == curr_pos:
            scroll_atmpt += 1
            if scroll_atmpt >= 3:
                scrolling = False
                break
            else:
                sleep(3)
        else:
            last_pos = curr_pos
            break

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: MicrosoftEdge=105.0.1343.27)
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF6FAB32352+17554]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF6FAACECA2+1438690]
	Ordinal0 [0x00007FF6FA6F9345+627525]
	Ordinal0 [0x00007FF6FA6FCA3A+641594]
	Ordinal0 [0x00007FF6FA6FC7BB+640955]
	Ordinal0 [0x00007FF6FA6FCB70+641904]
	Ordinal0 [0x00007FF6FA731E81+859777]
	Ordinal0 [0x00007FF6FA753C2A+998442]
	Ordinal0 [0x00007FF6FA72B144+831812]
	Ordinal0 [0x00007FF6FA753F30+999216]
	Ordinal0 [0x00007FF6FA768BF9+1084409]
	Ordinal0 [0x00007FF6FA753A43+997955]
	Ordinal0 [0x00007FF6FA7294C0+824512]
	Ordinal0 [0x00007FF6FA72A9F8+829944]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF6FA990588+134344]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF6FA979EB5+42485]


In [18]:
len(data)

2006

In [19]:
with open('QueenElizabeth.csv', 'w', newline='', encoding='utf-16') as f:
    header = ['UserName', 'Handle', 'TimeStamp','Tweet', 'Comments', 'Retweets', 'Likes' ]
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)